# <a id='toc1_'></a>[Threading](#toc0_)

Threading, Python'da aynı program içinde birden fazla iş parçacığı oluşturarak paralel işlem yapmamızı sağlayan bir modüldür. Her bir iş parçacığı, kendi işlem akışına sahip bağımsız çalışan küçük işlemlerdir. Threading, özellikle I/O yoğun işlemlerde, örneğin dosya okuma/yazma veya ağ işlemleri gibi durumlarda performansı artırmak için kullanılır.

In [1]:
import threading

**İçindekiler**<a id='toc0_'></a>    
- [Threading](#toc1_)    
  - [Threading Modülü ve Kullanımı](#toc1_1_)    
  - [Threading'de Dikkat Edilmesi Gerekenler](#toc1_2_)    
  - [Threading Sektörel Kullanımı](#toc1_3_)    
  - [Örnek: JSON Veritabanı](#toc1_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Threading Modülü ve Kullanımı](#toc0_)

Python'da threading için `threading` modülü kullanılır. Bu modülü kullanarak iş parçacıklarını oluşturabilir ve yönetebiliriz.

In [2]:
import time

def print_numbers():
    for i in range(1, 6):
        print(i)
        time.sleep(1)

def print_letters():
    for letter in 'ABCDE':
        print(letter)
        time.sleep(1)

if __name__ == "__main__":
    t1 = threading.Thread(target=print_numbers)
    t2 = threading.Thread(target=print_letters)

    t1.start()
    t2.start()

    t1.join()
    t2.join()

    print("Program sonlandı.")

1
A
2B

C3

D4

E
5
Program sonlandı.


Yukarıdaki örnekte, iki ayrı iş parçacığı oluşturduk. Birincisi `print_numbers` fonksiyonunu, diğeri ise `print_letters` fonksiyonunu işleyecektir. `start()` metoduyla her iki iş parçacığını da çalıştırdık. `join()` metodu ise her iş parçacığının tamamlanmasını beklememizi sağlar.

## <a id='toc1_2_'></a>[Threading'de Dikkat Edilmesi Gerekenler](#toc0_)

Threading kullanırken, aynı değişken veya veri kaynağına birden fazla iş parçacığı tarafından erişilebileceğini unutmayın. Bu durumda uygun senkronizasyon mekanizmaları kullanarak veri bütünlüğünü korumak önemlidir.

Threading, yalnızca I/O yoğun işlemler için uygun olabilir. Çünkü Python'da Global Interpreter Lock (GIL) mekanizması nedeniyle, CPU yoğun işlemlerde gerçek paralelizm elde etmek zor olabilir.

Threading, her zaman performansı artırmaz. İşlemler arasındaki geçiş maliyeti, bazı durumlarda threading'in performansını olumsuz yönde etkileyebilir.

## <a id='toc1_3_'></a>[Threading Sektörel Kullanımı](#toc0_)

Threading, Python'da genellikle I/O yoğun işlemler için kullanılır. Özellikle ağ işlemleri, dosya okuma/yazma işlemleri veya veritabanı işlemleri gibi durumlarda threading, uygulamanın tepkisini artırabilir ve zaman kazandırabilir.

Aynı program içinde birden fazla iş parçacığı oluşturarak paralel işlem yapmamızı sağlar. Özellikle I/O yoğun işlemlerde performansı artırabilir. Ancak GIL mekanizması nedeniyle CPU yoğun işlemlerde gerçek paralelizm elde etmek zor olabilir. Genellikle web scraping, ağ işlemleri ve dosya okuma/yazma gibi durumlarda sıklıkla kullanılır. Uygun senkronizasyon ve veri bütünlüğü sağlama önemlidir.

## <a id='toc1_4_'></a>[Örnek: JSON Veritabanı](#toc0_)

Bir müşteri veritabanı oluşturmak istiyoruz. Her müşterinin adını, e-posta adresini ve telefon numarasını kaydedeceğiz. Birden çok müşteri ekleme talebi eşzamanlı olarak gerçekleşebilir, bu nedenle threading kullanarak veritabanına ekleme işlemini paralel olarak yapacağız.

In [5]:
import threading
import json

def add_customer(database, name, email, phone):
    # Yeni müşteri verilerini sözlük olarak oluşturuyoruz
    new_customer = {
        'name': name,
        'email': email,
        'phone': phone
    }

    # Veritabanını güncellemek için threading kilidini kullanıyoruz
    with threading.Lock():
        # Önce mevcut verileri yüklüyoruz (eğer varsa)
        try:
            with open(database, 'r') as file:
                data = json.load(file)
        except FileNotFoundError:
            data = []

        # Yeni müşteriyi verilere ekliyoruz
        data.append(new_customer)

        # Verileri JSON dosyasına yazıyoruz
        with open(database, 'w') as file:
            json.dump(data, file)

if __name__ == "__main__":
    # Yeni müşteri eklemek için veritabanı dosyası
    DATABASE_FILE = "customers.json"

    # Eşzamanlı müşteri eklemek için iş parçacıkları oluşturuyoruz
    threads = []
    for i in range(5):  # Örneğin, 5 müşteri eklemek istiyoruz
        name = f"Customer {i}"
        email = f"customer{i}@example.com"
        phone = f"555-123-000{i:02}"  # Telefon numarasını örnek olarak ayarlıyoruz

        thread = threading.Thread(target=add_customer, args=(DATABASE_FILE, name, email, phone))
        thread.start()
        threads.append(thread)

    # Tüm iş parçacıklarının tamamlanmasını bekliyoruz
    for thread in threads:
        thread.join()

    # Son olarak, tüm müşteri verilerini yazdırıyoruz
    with open(DATABASE_FILE, 'r') as file:
        all_customers = json.load(file)
        print("Customer Database:")
        for customer in all_customers:
            print(f"Name: {customer['name']}, Email: {customer['email']}, Phone: {customer['phone']}")

Customer Database:
Name: Customer 0, Email: customer0@example.com, Phone: 555-123-00000
Name: Customer 1, Email: customer1@example.com, Phone: 555-123-00001
Name: Customer 4, Email: customer4@example.com, Phone: 555-123-00004


## Kaynakça
---

https://docs.python.org/tr/3/library/threading.html?highlight=threading#module-threading